## **Find the mean voxel spacing**

In [ ]:
import os
import nibabel as nib

# Specify the parent folder
parent_folder = "/kaggle/input/aneurysm-dataset-qingdao/aneurysm_dataset"

# Initialize variables for summation and count
voxel_sums = [0.0, 0.0, 0.0]
file_count = 0

# Iterate through each subfolder in the parent folder
for subfolder in os.listdir(parent_folder):
    subfolder_path = os.path.join(parent_folder, subfolder)
    
    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        label_file = os.path.join(subfolder_path, "aneurysms.nii")
        
        # Ensure the file exists
        if os.path.exists(label_file):
            # Load the NIfTI file
            label_nii = nib.load(label_file)
            
            # Get the header and voxel spacing
            header = label_nii.header
            voxel_spacing = header.get_zooms()[:3]  # x, y, z spacings
            
            # Update summation and count
            voxel_sums = [voxel_sums[i] + voxel_spacing[i] for i in range(3)]
            file_count += 1

# Calculate mean voxel spacing
# if file_count > 0:
#     mean_voxel_spacing = [voxel_sums[i] / file_count for i in range(3)]
#     print("Mean Voxel Spacing (x, y, z):", mean_voxel_spacing)
# else:
#     print("No valid files found.")


Mean Voxel Spacing (x, y, z): [0.35236164320886665, 0.35236164320886665, 0.5446875839106804]


## **Process the data**

## Rescale

In [3]:
import numpy as np
from scipy.ndimage import zoom

def rescale(image, label, input_space, output_space=(0.35236164320886665, 0.35236164320886665, 0.5446875839106804)):
    assert image.shape == label.shape, f"Image shape: {image.shape} != Label shape: {label.shape}"
    zoom_factor = tuple(input_space[i] / output_space[i] for i in range(3))
    # Rescale the image with cubic interpolation
    image_rescale = zoom(image, zoom_factor, order=3)
    # Rescale the label with nearest-neighbor interpolation
    label_rescale = zoom(label, zoom_factor, order=0)
    return image_rescale, label_rescale

## Normalize

In [4]:
def normalize_data(image_data):
  mean = np.mean(image_data)
  std = np.std(image_data)
  normalized_data = (image_data - mean) / std
  return normalized_data


# **Write to txt**

In [5]:
import os
import numpy as np

def save_array_as_file(array, filename):
    """
    Saves a single NumPy array to a .npy file, ensuring the directory exists.

    Args:
      array: NumPy array to save.
      filename: Full path including the file name to save.
    """
    # Ensure the directory exists
    directory = os.path.dirname(filename)
    os.makedirs(directory, exist_ok=True)  # Create directories if they don't exist

    # Save array to file
    np.save(filename, array)
    print(f"Saved array to {filename}")


In [ ]:

import os
import nibabel as nib
import numpy as np
from scipy.ndimage import label, measurements, find_objects
parent_folder = "C:/Users/XgearHN_Lap/Downloads/nii_aneurysm_dataset"
# trained_folders = ['10055F', '10020', '10065B', '10052B', '10025', '10076B', '10016', '10050B', '10023', '10064F', '10034', '10069B', '10017', '10044B', '10049F', '10073B', '10051F', '10029', '10071B', '10061F', '10013', '10070B', '10015', '10054F', '10047B', '10066B', '10067F', '10055B', '10010', '10039', '10005', '10008', '10032', '10062B', '10077B', '10057F', '10044F', '10072F', '10009', '10004', '10001', '10002', '10006', '10018', '10024', '10033', '10035', '10038', '10047F', '10048B', '10060B', '10062F', '10063B', '10065F', '10068F', '10073F', '10074B', '10075F', '10076F', '10078F', '10007', '10022', '10026', '10028', '10036', '10041', '10045B', '10046F', '10050F', '10051B', '10056F', '10057B', '10059B', '10061B', '10064B', '10067B', '10069F', '10075B', '10077F', '10078B', '10037', '10053F', '10048F', '10054B', '10052F', '10011', '10019', '10040', '10056B', '10068B', '10045F', '10072B', '10049B', '10031', '10060F', '10043', '10027']
for subfolder in os.listdir(parent_folder):
    # if i == 20:
    #     break
    # if str(subfolder) not in trained_folders:
    #     i+=1
    #     trained_folders.append(str(subfolder))
        subfolder_path = os.path.join(parent_folder, subfolder)
        
                # Check if it's a directory
        if os.path.isdir(subfolder_path):
            label_file = os.path.join(subfolder_path, "aneurysms.nii")
            label_nii_data = nib.load(label_file)
            label_data = label_nii_data.get_fdata()
            label_data[label_data==2] = 0
        
            print(str(subfolder))
            header = label_nii_data.header
                # Get voxel spacing (pixdim[1:4] contains voxel dimensions along x, y, z)
            voxel_spacing = header.get_zooms()
                                
            image_file = os.path.join(subfolder_path, "pre/TOF.nii")
            image_nii_data = nib.load(image_file)
            image_data = image_nii_data.get_fdata()
                    
            image_rescale, label_rescale = rescale(image_data, label_data, voxel_spacing)
                #normalize image
            normalized_image = normalize_data(image_rescale)
            save_array_as_file(normalized_image, f'C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset/{subfolder}/image.npy')
            save_array_as_file(label_rescale, f'C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset/{subfolder}/label.npy')


In [13]:
import os
import numpy as np
from scipy.ndimage import label, find_objects

def find_bounding_boxes_3d(label_path):
    """Find 3D bounding boxes from a 3D label.npy file."""
    label_data = np.load(label_path)  # Load 3D label.npy
    labeled_array, num_features = label(label_data)  # Label connected components
    objects = find_objects(labeled_array)  # Get bounding boxes

    bounding_boxes = []
    for obj in objects:
        if obj is not None:
            # Extract bounding box coordinates (zmin, zmax, ymin, ymax, xmin, xmax)
            xmin, xmax = obj[0].start, obj[0].stop
            ymin, ymax = obj[1].start, obj[1].stop
            zmin, zmax = obj[2].start, obj[2].stop
            bounding_boxes.append((xmin, ymin, zmin, xmax, ymax, zmax))

    return bounding_boxes

def process_subfolders(parent_folder):
    """Process each subfolder to find 3D bounding boxes and save to file."""
    for subdir in os.listdir(parent_folder):
        subdir_path = os.path.join(parent_folder, subdir)
        if os.path.isdir(subdir_path):  # Check if it's a directory
            label_path = os.path.join(subdir_path, "label.npy")
            if os.path.exists(label_path):  # Check if label.npy exists
                bounding_boxes = find_bounding_boxes_3d(label_path)

                # Write to bounding_boxes.txt
                bounding_boxes_path = os.path.join(subdir_path, "bounding_boxes.txt")
                with open(bounding_boxes_path, "w") as f:
                    pass
                with open(bounding_boxes_path, "a") as f:  # Append to existing file if it exists:
                    for i, bbox in enumerate(bounding_boxes):
                        f.write(f"Bounding Box {i} Coordinates:\n")
                        f.write(f"x: {bbox[0]}-{bbox[3]}\n")
                        f.write(f"y: {bbox[1]}-{bbox[4]}\n")
                        f.write(f"z: {bbox[2]}-{bbox[5]}\n")
                print(f"Bounding boxes written to: {bounding_boxes_path}")

# Change this to the path of your parent folder
parent_folder = "C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset"
process_subfolders(parent_folder)


Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10001\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10002\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10003\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10004\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10005\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10006\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10007\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10008\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_aneurysm_dataset\10009\bounding_boxes.txt
Bounding boxes written to: C:/Users/XgearHN_Lap/Downloads/pre_an